In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
skip = pd.read_csv('data/skip.csv', header = None)
normal = pd.read_csv('data/normal.csv', header = None)
print(skip)
print(normal)

         0     1     2       3       4       5
0     1.93  0.32  0.76  409.25  265.14  -80.20
1     1.97 -0.02  1.00  485.79  236.46  -76.05
2     1.97 -0.44  1.25  557.08  194.83  -72.21
3     1.97 -0.98  1.56  595.36  148.32  -81.30
4     1.97 -1.54  1.93  575.88  116.28 -101.81
...    ...   ...   ...     ...     ...     ...
2995  0.48 -1.82 -0.36 -295.60 -490.43  -97.11
2996  0.36 -1.70 -0.61 -256.42 -489.09  -55.48
2997  0.28 -1.56 -0.80 -228.46 -460.34  -20.51
2998  0.19 -1.46 -0.96 -207.10 -418.59    4.52
2999  0.08 -1.40 -1.16 -187.32 -365.67   24.72

[3000 rows x 6 columns]
         0     1     2       3       4       5
0    -1.60  0.16  1.47  210.88 -187.38 -425.61
1     1.98  0.72  1.61  208.69 -124.03 -365.92
2     1.97  1.04  1.60  205.27  -62.14 -274.79
3     1.97  1.07  1.54  185.43   -1.04 -177.31
4    -1.92  0.82  1.42  153.14   49.68  -89.72
...    ...   ...   ...     ...     ...     ...
2995  0.76  0.19  1.02  -26.43  -44.92    3.72
2996  0.77  0.24  1.14  -37.42  -51

In [3]:
SAMPLES_PER_GESTURE = 30

In [4]:
def processData(d, v):
    dataX = np.empty([0,SAMPLES_PER_GESTURE*6])
    dataY = np.empty([0])

    data  = d.values
    dataNum = data.shape[0] // SAMPLES_PER_GESTURE


    for i in tqdm(range(dataNum)):
        tmp = []
        for j in range(SAMPLES_PER_GESTURE):  # 正规化
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][0] + 2.0) / 4.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][1] + 2.0) / 4.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][2] + 2.0) / 4.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][3] + 2000.0) / 4000.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][4] + 2000.0) / 4000.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][5] + 2000.0) / 4000.0]

        tmp = np.array(tmp)

        tmp = np.expand_dims(tmp, axis = 0)

        dataX = np.concatenate((dataX, tmp), axis = 0)
        dataY = np.append(dataY, v)

    return dataX, dataY

In [5]:
skipX, skipY = processData(skip, 0)
normalX, normalY = processData(normal, 1)
dataX = np.concatenate((skipX, normalX), axis = 0)
dataY = np.concatenate((skipY, normalY), axis = 0)

100%|██████████| 100/100 [00:00<00:00, 4350.08it/s]


In [6]:
permutationTrain = np.random.permutation(dataX.shape[0])
print(permutationTrain)

[196 181  12 193  34  43  98 197 139 171  66  51  60 109 176 156 143  49
 137 177  84  90  55 175 195  59   2 110 190 151 122 169  70  31 150  94
 198 184 154  39  45  23  67 121  92  15 144 178  35 105 188  54  48 141
 155   8  16 159  22  63  26 107  10 185  30  56  27  91 138 118  76  57
  81 163 136  20  74 180   5 112 119  82  17 189   1 113 153 101  89 199
  24 166 148   9  75 182  99 142   3 152 145 114 191   7 186  78  61  62
  96 102 104 187 140  73   6 174 162 123 120 128  42  19 127  11 116 106
 167  85  41 133 172  93 135  38 125 157  68  18 183  87 129  95  25 103
   4 134  37  46  33 164  53  40  86 165  47  28  97  80  58 132   0 194
  50 100  36 192 170 161  64  72  32 117  29  21  69  88  14 146 168  83
 130 124 160  13 179 126  52  65 147  79 173 131  71 108  44  77 149 111
 158 115]


In [7]:
vfoldSize = int(dataX.shape[0]/100*20)

xTest = dataX[0:vfoldSize]
yTest = dataY[0:vfoldSize]

xTrain = dataX[vfoldSize:dataX.shape[0]]
yTrain = dataY[vfoldSize:dataY.shape[0]]

In [8]:
model = keras.Sequential()
model.add(keras.layers.Dense(32, input_shape =(6*SAMPLES_PER_GESTURE,), activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))
adam = keras.optimizers.Adam()

In [9]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=adam,
              metrics=['sparse_categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                5792      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 6,354
Trainable params: 6,354
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(xTrain, yTrain, batch_size=1, validation_data=(xTest, yTest), epochs=200, verbose=1)

Train on 160 samples, validate on 40 samples
Epoch 1/200
160/160 [==============================] - 2s 10ms/sample - loss: 0.2666 - sparse_categorical_accuracy: 0.9688 - val_loss: 0.1580 - val_sparse_categorical_accuracy: 1.0000
Epoch 2/200
160/160 [==============================] - 0s 3ms/sample - loss: 0.0451 - sparse_categorical_accuracy: 0.9937 - val_loss: 0.0292 - val_sparse_categorical_accuracy: 1.0000
Epoch 3/200
160/160 [==============================] - 1s 3ms/sample - loss: 0.0407 - sparse_categorical_accuracy: 0.9812 - val_loss: 0.0107 - val_sparse_categorical_accuracy: 1.0000
Epoch 4/200
160/160 [==============================] - 1s 3ms/sample - loss: 0.0427 - sparse_categorical_accuracy: 0.9812 - val_loss: 0.0117 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/200
160/160 [==============================] - 1s 4ms/sample - loss: 0.0072 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0413 - val_sparse_categorical_accuracy: 1.0000
Epoch 6/200
160/160 [==================

In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

open("model", "wb").write(tflite_model)

26996

In [12]:
!xxd -i model >> model.h